In [1]:
import pandas as pd
from itertools import groupby
from operator import itemgetter

In [2]:
df = pd.read_csv('../rawdata/user_profiles_participation.csv')

In [3]:
df.columns = ['Years', 'Years Label', 'index', 'url']
df = df[['Years', 'url']]
df.head()

,Years,url
0,NaN,http://nanowrimo.org/participants/nicaless
1,2014 winner,http://nanowrimo.org/participants/nicaless
2,2015 participant,http://nanowrimo.org/participants/nicaless
3,2005 winner,http://nanowrimo.org/participants/rachel-b-moore
4,2006 winner,http://nanowrimo.org/participants/rachel-b-moore


In [4]:
years_new_list = []
for i in df['Years']:
    if isinstance(i, float):
        years_new_list.append("donor")
    else:
        years_new_list.append(i.split(" "))
        

In [5]:
years_new_list


['donor',
 ['2014', 'winner'],
 ['2015', 'participant'],
 ['2005', 'winner'],
 ['2006', 'winner'],
 'donor',
 ['2007', 'winner'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'winner'],
 'donor',
 ['2010', 'winner'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'participant'],
 'donor',
 ['2013', 'participant'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 'donor',
 ['2015', 'participant'],
 ['2004', 'winner'],
 ['2005', 'winner'],
 ['2006', 'participant'],
 'donor',
 ['2007', 'participant'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'participant'],
 ['2010', 'participant'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'winner'],
 'donor',
 ['2013', 'winner'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 ['2012', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 ['2012', 'participant'],
 ['2013', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 'donor',
 ['2015', 'winner'],
 ['2015', '

In [6]:
df['Years'] = years_new_list

In [7]:
writernames = pd.read_csv('../clean data/user_summary.csv')

In [8]:
writernames = writernames[['Writer Name', 'url']]
writernames.head()

,Writer Name,url
0,Nicaless,http://nanowrimo.org/participants/nicaless
1,Rachel B. Moore,http://nanowrimo.org/participants/rachel-b-moore
2,abookishbabe,http://nanowrimo.org/participants/abookishbabe
3,alexabexis,http://nanowrimo.org/participants/alexabexis
4,AllYellowFlowers,http://nanowrimo.org/participants/AllYellowFlo...


In [9]:
new_df = pd.merge(writernames, df, on='url')
new_df = new_df[['Writer Name', 'Years']]
new_df.head()

,Writer Name,Years
0,Nicaless,donor
1,Nicaless,"[2014, winner]"
2,Nicaless,"[2015, participant]"
3,Rachel B. Moore,"[2005, winner]"
4,Rachel B. Moore,"[2006, winner]"


In [10]:
allnames = writernames[['Writer Name']]
allnames.head()

,Writer Name
0,Nicaless
1,Rachel B. Moore
2,abookishbabe
3,alexabexis
4,AllYellowFlowers


In [11]:
groupnames = new_df.groupby('Writer Name')

In [15]:
list_dics = []
test = ["alexabexis"]
#for name in test:
for name in allnames['Writer Name']:
    group = groupnames.get_group(name)
    dic = {}
    donor_years = []
    win_years = []
    part_years = []
    
    for i in group.index:
        if group['Years'][i] == 'donor':
            donor_years.append(group['Years'][i+1][0])
        elif group['Years'][i][1] == "winner":

            win_years.append(group['Years'][i][0])
            part_years.append(group['Years'][i][0])
        else:
            part_years.append(group['Years'][i][0])
    
    donor_years = map(int, donor_years)
    consec_donor = []
    for k, g in groupby(enumerate(donor_years), lambda (i, x): i-x):
        consec_donor.append(len(map(itemgetter(1), g)))
        
    win_years = map(int, win_years)
    consec_wins = []
    for k, g in groupby(enumerate(win_years), lambda (i, x): i-x):
        consec_wins.append(len(map(itemgetter(1), g)))
    
    part_years = map(int, part_years)
    consec_part = []
    for k, g in groupby(enumerate(part_years), lambda (i, x): i-x):
        consec_part.append(len(map(itemgetter(1), g)))
    
    dic['Writer Name'] = name
    dic['Wins'] = sum(consec_wins)
    dic['Participated'] = sum(consec_part)
    dic['Donations'] = sum(consec_donor)
    if len(consec_donor) < 1:
        dic['Consecutive Donor'] = 0
    else:
        dic['Consecutive Donor'] = max(consec_donor)
    if len(consec_wins) < 1:
        dic['Consecutive Wins'] = 0
    else:
        dic['Consecutive Wins'] = max(consec_wins)
    if len(consec_part) < 1:
        dic['Consecutive Part'] = 0
    else:
        dic['Consecutive Part'] = max(consec_part)

    dic['Donor Years'] = donor_years
    dic['Win Years'] = win_years
    dic['Part Years'] = part_years
    list_dics.append(dic)

In [31]:
final_df = pd.DataFrame(list_dics)
final_df.head()

,Consecutive Donor,Consecutive Part,Consecutive Wins,Donations,Donor Years,Part Years,Participated,Win Years,Wins,Writer Name
0,1,2,1,1,[2014],"[2014, 2015]",2,[2014],1,Nicaless
1,9,11,7,9,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",11,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",9,Rachel B. Moore
2,1,1,0,1,[2015],[2015],1,[],0,abookishbabe
3,5,12,5,8,"[2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015]","[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...",12,"[2004, 2005, 2008, 2011, 2012, 2013, 2014, 2015]",8,alexabexis
4,0,2,0,0,[],"[2012, 2014, 2015]",3,[],0,AllYellowFlowers


In [32]:
final_df = final_df[['Writer Name', 'Wins', 'Donations', 'Participated', 'Consecutive Donor', 'Consecutive Wins', 'Consecutive Part', 'Part Years', 'Win Years', "Donor Years"]]

In [33]:
final_df.to_csv("../clean data/user_participation_clean.csv")
final_df = final_df[['Wins', 'Donations', 'Participated', 'Consecutive Donor', 'Consecutive Wins', 'Consecutive Part', 'Part Years', 'Win Years', "Donor Years"]]

In [35]:
summary = pd.read_csv("../clean data/user_summary.csv")
del summary['Unnamed: 0']
del summary['Consecutive Donor']
del summary['Consecutive Wins']
del summary['Consecutive Part']
del summary['Part Years']
del summary['Win Years']
del summary["Donor Years"]
summary.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs'],
      dtype='object')

In [36]:
summary = pd.concat([summary, final_df], axis = 1)

In [38]:
summary.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs', u'Wins',
       u'Donations', u'Participated', u'Consecutive Donor',
       u'Consecutive Wins', u'Consecutive Part', u'Part Years', u'Win Years',
       u'Donor Years'],
      dtype='object')

In [39]:
summary.to_csv("../clean data/user_summary.csv")

In [41]:
summary.head()

,Writer Name,Member Length,LifetimeWordCount,url,Age,Birthday,Favorite books or authors,Favorite noveling music,Hobbies,Location,...,Expected Consec Subs,Wins,Donations,Participated,Consecutive Donor,Consecutive Wins,Consecutive Part,Part Years,Win Years,Donor Years
0,Nicaless,2,"75,148",http://nanowrimo.org/participants/nicaless,24,December 20,"Ursula Le Guin, J.K.","Classical, Musicals","Reading, Video Games, Blogging, Learning","San Francisco, CA",...,12.000000,1,1,2,1,1,2,"[2014, 2015]",[2014],[2014]
1,Rachel B. Moore,10,"528,693",http://nanowrimo.org/participants/rachel-b-moore,NaN,NaN,"2666, Unaccustomed Earth, Exit Music, Crazy Lo...","Belle and Sebastian, Elliott Smith, PJ Harvey,...","Reading, volunteering, knitting, listening to ...",San Francisco,...,8.727273,9,9,11,9,7,11,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201..."
2,abookishbabe,1,"29,299",http://nanowrimo.org/participants/abookishbabe,NaN,April 2,"Colleen Hoover, Veronica Roth, Jennifer Niven,...",Tori Kelley,"Reading (DUH), Day dreaming, Going to Disneyla...","Sacramento, CA",...,1.000000,0,1,1,1,0,1,[2015],[],[2015]
3,alexabexis,11,"554,300",http://nanowrimo.org/participants/alexabexis,NaN,NaN,NaN,Three Goddesses playlist Florence + the Machin...,"drawing, reading, movies & TV shows, comics, p...",New York City,...,7.083333,8,8,12,5,5,12,"[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...","[2004, 2005, 2008, 2011, 2012, 2013, 2014, 2015]","[2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015]"
4,AllYellowFlowers,3,"41,428",http://nanowrimo.org/participants/AllYellowFlo...,NaN,NaN,"Lolita, Jesus' Son, Ask the",the sound of the coffeemaker,cryptozoology,Allston,...,19.000000,0,0,3,0,0,2,"[2012, 2014, 2015]",[],[]
